In [1]:
import geopandas as gpd
import folium
from IPython.display import display
import pandas as pd

In [4]:
shapefile_paths = [
    "../MSS_nodes/dsm2_nodes_newcs_extranodes.shp",
    "../fc2024.01_chan/FC2024.01_channels_centerlines.shp"
]

In [5]:
nodes = gpd.read_file(shapefile_paths[0])
nodes.head

<bound method NDFrame.head of                X          Y   id                        geometry
0    586524.5000  4214278.5  329      POINT (586524.5 4214278.5)
1    595501.8125  4215164.0  328      POINT (595501.812 4215164)
2    587823.0625  4224132.5  327    POINT (587823.062 4224132.5)
3    611764.3125  4234500.0  326      POINT (611764.312 4234500)
4    609429.0625  4235459.0  325      POINT (609429.062 4235459)
..           ...        ...  ...                             ...
473          NaN        NaN  811   POINT (640955.51 4185200.504)
474          NaN        NaN  812  POINT (640236.914 4185018.152)
475          NaN        NaN  814  POINT (638627.967 4184416.817)
476          NaN        NaN  813  POINT (639130.486 4183087.804)
477          NaN        NaN  815  POINT (631626.258 4183226.381)

[478 rows x 4 columns]>

In [12]:
channels = gpd.read_file(shapefile_paths[1])
channels.head

<bound method NDFrame.head of       id                                           geometry
0      1  LINESTRING (652291.736 4172448.043, 652209.821...
1      2  LINESTRING (652401.235 4174797.442, 652581.296...
2      3  LINESTRING (651808.247 4176015.422, 651766.413...
3      4  LINESTRING (650024.633 4176799.368, 649933.498...
4      5  LINESTRING (648990.218 4180561.514, 648949.832...
..   ...                                                ...
546  819  LINESTRING (640236.667 4185024.396, 639714.773...
547  820  LINESTRING (639153.103 4183088.611, 639153.56 ...
548  821  LINESTRING (638934.638 4184888.455, 638822.624...
549  822  LINESTRING (638615.055 4184423.33, 638562.21 4...
550  823  LINESTRING (631626.258 4183226.381, 631559.926...

[551 rows x 2 columns]>

In [5]:
# nodes to highlight, based on figure 

nodes_to_highlight = [112, 176, 69]
nodes_filter = nodes[nodes['id'].isin(nodes_to_highlight)]
nodes_filter.head

<bound method NDFrame.head of                X           Y   id                       geometry
134  632155.4375  4186851.25  176  POINT (632155.438 4186851.25)
207  634764.4375  4193850.50  112   POINT (634764.438 4193850.5)
287  628267.0000  4185764.00   69         POINT (628267 4185764)>

In [29]:
# read in channel dataframe 

channels_with_numbers = pd.read_csv('../channel_names_from_h5.csv')
channels_with_numbers = channels_with_numbers.rename(columns={'chan_no': 'id'})
channels_with_numbers.head

<bound method NDFrame.head of      Unnamed: 0    name   id distance variable interval period_op  \
0             0     ANH   52        0     flow    15MIN      inst   
1             1     ANH   52        0    stage    15MIN      inst   
2             2     BDT   11       81     flow    15MIN      inst   
3             3     BDT   11       81    stage    15MIN      inst   
4             4  CCF_up  232      500    stage    15MIN      inst   
..          ...     ...  ...      ...      ...      ...       ...   
156         156     VNS   17     4706    stage    15MIN      inst   
157         157     VON  410        0     flow    15MIN      inst   
158         158     VON  410        0    stage    15MIN      inst   
159         159     WCI  232      457     flow    15MIN      inst   
160         160     WCI  232      457    stage    15MIN      inst   

                          file  
0    ./output/FPV2Mb_hydro.dss  
1    ./output/FPV2Mb_hydro.dss  
2    ./output/FPV2Mb_hydro.dss  
3    ./ou

In [30]:
channels_merge = pd.merge(
    channels,
    channels_with_numbers,
    how='left',
    left_on='id',
    right_on='id'
)
channels_merge.head

<bound method NDFrame.head of       id                                           geometry  Unnamed: 0 name  \
0      1  LINESTRING (652291.736 4172448.043, 652209.821...         NaN  NaN   
1      2  LINESTRING (652401.235 4174797.442, 652581.296...         NaN  NaN   
2      3  LINESTRING (651808.247 4176015.422, 651766.413...         NaN  NaN   
3      4  LINESTRING (650024.633 4176799.368, 649933.498...        28.0  DVI   
4      4  LINESTRING (650024.633 4176799.368, 649933.498...        29.0  DVI   
..   ...                                                ...         ...  ...   
645  820  LINESTRING (639153.103 4183088.611, 639153.56 ...       136.0  SUR   
646  820  LINESTRING (639153.103 4183088.611, 639153.56 ...       137.0  SUR   
647  821  LINESTRING (638934.638 4184888.455, 638822.624...         NaN  NaN   
648  822  LINESTRING (638615.055 4184423.33, 638562.21 4...         NaN  NaN   
649  823  LINESTRING (631626.258 4183226.381, 631559.926...         NaN  NaN   

    dista

In [31]:
# Ensure the CRS is EPSG:4326 (latitude/longitude)
channels_merge = channels_merge.to_crs(epsg=4326)

# Filter for specific IDs
filtered_channels = channels_merge[channels_merge['id'].isin([211, 79, 134])]

# Initialize the map centered on the average location of the filtered geometries
centroid = filtered_channels.geometry.unary_union.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=12)

# Add polylines for the filtered LineStrings
for _, row in filtered_channels.iterrows():
    popup_content = f"""
    id = {row['id']}<br>
    name = {row.get('name', 'N/A')}<br>
    distance = {row.get('distance', 'N/A')}<br>
    variable = {row.get('variable', 'N/A')}<br>
    interval = {row.get('interval', 'N/A')}<br>
    period_op = {row.get('period_op', 'N/A')}
    """
    
    # Extract coordinates for the LineString
    coordinates = [(point[1], point[0]) for point in row.geometry.coords]  # Convert (x, y) to (lat, lon)
    
    # Add the LineString to the map
    folium.PolyLine(
        locations=coordinates,
        color='darkgreen',
        popup=folium.Popup(popup_content, max_width=300)
    ).add_to(m)

# Display the map
m

/var/folders/45/p22_x33n4s1dbmnzjdjzfr_80000gn/T/ipykernel_3392/4019552048.py:8: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = filtered_channels.geometry.unary_union.centroid


In [6]:
import geopandas as gpd
import folium
from IPython.display import display

def create_multi_layer_map(shapefile_paths, filtered_gdf=None):
    """
    Creates an interactive map with multiple shapefile layers and optional filtered points in red.
    
    Parameters:
    - shapefile_paths: List of file paths to the shapefiles.
    - filtered_gdf: A GeoDataFrame with filtered points to display in red (optional).
    """
    # Initialize a list to store valid GeoDataFrames and their centroids
    gdfs = []
    all_centroids = []
    
    for path in shapefile_paths:
        # Read shapefile
        gdf = gpd.read_file(path)
        
        # Ensure the CRS is EPSG:4326
        if gdf.crs != "EPSG:4326":
            gdf = gdf.to_crs("EPSG:4326")
        
        # Ensure filtered GeoDataFrame is in the correct CRS
        if filtered_gdf is not None and filtered_gdf.crs != "EPSG:4326":
            filtered_gdf = filtered_gdf.to_crs("EPSG:4326")
        
        # Check if there are valid geometries
        if gdf.is_empty.any():
            print(f"Warning: Some geometries in {path} are empty and will be ignored.")
            gdf = gdf[~gdf.is_empty]  # Remove empty geometries
        
        # Compute the centroid in EPSG:4326 (no need for reprojection)
        centroid = gdf.geometry.centroid
        center = [centroid.y.mean(), centroid.x.mean()]
        
        # Append to the list
        gdfs.append((gdf, center))
        all_centroids.append(center)
    
    # Calculate map center based on all layers
    avg_lat = sum([c[0] for c in all_centroids]) / len(all_centroids)
    avg_lon = sum([c[1] for c in all_centroids]) / len(all_centroids)
    
    # Create a Folium map centered on the average centroid
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=10)
    
    # Add each shapefile as a separate layer to the map
    for idx, (gdf, _) in enumerate(gdfs):
        folium.GeoJson(
            gdf,
            name=f"Layer {idx + 1}: {shapefile_paths[idx].split('/')[-1]}"
        ).add_to(m)
    
    # If filtered_gdf is provided, add the filtered points in red
    if filtered_gdf is not None:
        for _, row in filtered_gdf.iterrows():
            folium.Marker(
                location=[row['geometry'].y, row['geometry'].x], 
                popup=f"ID: {row['id']}", 
                icon=folium.Icon(color='red', icon="circle")
            ).add_to(m)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    # Display the map in the notebook
    display(m)


In [7]:

create_multi_layer_map(shapefile_paths, nodes_filter)

/var/folders/45/p22_x33n4s1dbmnzjdjzfr_80000gn/T/ipykernel_3392/1209274723.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid
/var/folders/45/p22_x33n4s1dbmnzjdjzfr_80000gn/T/ipykernel_3392/1209274723.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid
